In [1]:
import sys
sys.path.append('../1_format/')
sys.path.append('../2_analyse/')

In [2]:
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn
import numpy as np
import glob
import geopandas
from matplotlib.pyplot import figure
seaborn.set()

In [3]:
pd.set_option('display.max_columns', None)

### Format BODACC

In [7]:
df_bodacc = pd.read_csv('../0_data/annonces-commerciales.csv', 
                        usecols=['id', 'dateparution', 'numeroannonce', 'jugement', 'numerodepartement', 'region_code',
                                 'tribunal', 'commercant', 'ville', 'registre', 'listepersonnes'],
                        sep=';')

In [9]:
df_bodacc = df_bodacc.dropna(subset='listepersonnes')


In [29]:
df_bodacc['jugement'].iloc[0]

'{"date": "19 mai 2010", "complementJugement": "Jugement prononçant la liquidation judiciaire, date de cessation des paiements le 30 septembre 2007 désignant liquidateur Scp Philippe Delaere, boulevard Docteur Chevrel, Immeuble Constens, 44500 La Baule Escoublac. Les déclarations de créances sont à déposer au liquidateur dans les deux mois de la présente publication.", "type": "initial", "famille": "Jugement d\'ouverture", "nature": "Jugement d\'ouverture de liquidation judiciaire"}'

In [13]:
s = df_bodacc.explode('listepersonnes', ignore_index=True)

In [22]:
from pandas import json_normalize
df = json_normalize(df_bodacc)


In [24]:
df_bodacc

,id,dateparution,numeroannonce,numerodepartement,region_code,tribunal,commercant,ville,registre,listepersonnes,jugement
0,A201001061332,2010-03-06,1332,44,52.0,TRIBUNAL DE COMMERCE DE SAINT-NAZAIRE,"VOILERIE SWING, PERRAIS, Frédéric",Pornichet,"448 800 516,448800516","{""personne"": {""typePersonne"": ""pp"", ""nom"": ""PE...","{""date"": ""19 mai 2010"", ""complementJugement"": ..."
1,A201001061337,2010-03-06,1337,49,52.0,TRIBUNAL DE GRANDE INSTANCE D'ANGERS,"GARROS, Marie-Anne",Liré,NaN,"{""personne"": {""typePersonne"": ""pp"", ""nom"": ""GA...","{""date"": ""26 octobre 2009"", ""complementJugemen..."
2,A201001061344,2010-03-06,1344,49,52.0,TRIBUNAL DE COMMERCE D'ANGERS,LES ARTISANS DU LITTORAL,Les Rosiers sur Loire,"510 397 144,510397144","{""personne"": {""typePersonne"": ""pm"", ""activite""...","{""date"": ""12 mai 2010"", ""complementJugement"": ..."
3,A201001061357,2010-03-06,1357,49,52.0,TRIBUNAL DE COMMERCE D'ANGERS,SARL ROMUALD NAVEAU,Trélazé,"478 440 118,478440118","{""personne"": {""typePersonne"": ""pm"", ""activite""...","{""date"": ""12 mai 2010"", ""complementJugement"": ..."
4,A20100106926,2010-03-06,926,11,76.0,TRIBUNAL DE COMMERCE DE CARCASSONNE,SOCATIS SARL,Villepinte,"503 260 218,503260218","{""personne"": {""typePersonne"": ""pm"", ""activite""...","{""date"": ""17 mai 2010"", ""complementJugement"": ..."
...,...,...,...,...,...,...,...,...,...,...,...
2730630,A202301002262,2023-05-25,2262,83,93.0,Greffe du Tribunal de Commerce de Fréjus,L'ESCALE LES CH'TI,Gassin,"818842379,818 842 379","{""personne"": {""typePersonne"": ""pm"", ""numeroImm...","{""type"": ""initial"", ""famille"": ""Jugement de cl..."
2730631,A202301002270,2023-05-25,2270,84,93.0,Greffe du Tribunal de Commerce d'Avignon,G2M TRANSPORTS,Mornas,"853643674,853 643 674","{""personne"": {""typePersonne"": ""pm"", ""numeroImm...","{""type"": ""initial"", ""famille"": ""Avis de dépôt""..."
2730632,A202301002280,2023-05-25,2280,91,11.0,Greffe du Tribunal de Commerce d'Evry,ASSOCIATION DES CINEMAS INDEPENDANTS,Boussy-Saint-Antoine,"450624192,450 624 192","{""personne"": {""typePersonne"": ""pm"", ""numeroImm...","{""type"": ""initial"", ""famille"": ""Jugement prono..."
2730633,A202301002293,2023-05-25,2293,92,11.0,Greffe du Tribunal de Commerce de Nanterre,NAS 92,Asnières-sur-Seine,"893263277,893 263 277","{""personne"": {""typePersonne"": ""pm"", ""numeroImm...","{""type"": ""initial"", ""famille"": ""Jugement d'ouv..."


In [ ]:
df_bodacc['jugement_nature'] = df_bodacc['jugement'].apply(lambda x: json.loads(x)['nature'])
df_bodacc['jugement_famille'] = df_bodacc['jugement'].apply(lambda x: json.loads(x)['famille'])
df_bodacc['jugement_date'] = df_bodacc['jugement'].apply(lambda x: json.loads(x)['date'] if 'date' in json.loads(x) else '')
df_bodacc['jugement_complement'] = df_bodacc['jugement'].apply(lambda x: json.loads(x)['complementJugement'] 
                                                               if 'complementJugement' in json.loads(x) else '')
df_bodacc = df_bodacc.drop('jugement', axis=1)

In [5]:
df_bodacc.isna().sum()

id                        0
dateparution              0
numeroannonce             0
numerodepartement         0
region_code              15
tribunal                  0
commercant              552
ville                  5498
registre             277610
jugement               1599
dtype: int64

In [56]:
# get jugement info
df_bodacc = df_bodacc.dropna(subset='jugement')
df_bodacc['jugement_nature'] = df_bodacc['jugement'].apply(lambda x: json.loads(x)['nature'])
df_bodacc['jugement_famille'] = df_bodacc['jugement'].apply(lambda x: json.loads(x)['famille'])
df_bodacc['jugement_date'] = df_bodacc['jugement'].apply(lambda x: json.loads(x)['date'] if 'date' in json.loads(x) else '')
df_bodacc['jugement_complement'] = df_bodacc['jugement'].apply(lambda x: json.loads(x)['complementJugement'] 
                                                               if 'complementJugement' in json.loads(x) else '')
df_bodacc = df_bodacc.drop('jugement', axis=1)

In [57]:
# fix error import
df_bodacc['jugement_famille'] = df_bodacc['jugement_famille'].replace('Jugement prononÃ§ant', 'Jugement prononçant')
df_bodacc['jugement_famille'] = df_bodacc['jugement_famille'].replace('Avis de dÃ©pÃ´t', 'Avis de dépôt')
df_bodacc['jugement_famille'] = df_bodacc['jugement_famille'].replace('Jugement de clÃ´ture', 'Jugement de clôture')
df_bodacc['jugement_famille'] = df_bodacc['jugement_famille'].replace("ArrÃªt de la Cour d'Appel", "Arrêt de la Cour d'Appel")

In [58]:
# get rid of null siren with no info
df_bodacc = df_bodacc.dropna(subset='registre')
df_bodacc['registre'] = df_bodacc['registre'].astype(str)
df_bodacc['registre'] = df_bodacc['registre'].apply(lambda x : x.replace('000 000 000,000000000,', ''))
df_bodacc = df_bodacc[df_bodacc['registre'] != '000 000 000,000000000']

In [59]:
# extract SIREN
df_bodacc['SIREN'] = df_bodacc['registre'].apply(lambda x: x.split(',')[0].replace(' ', ''))
df_bodacc = df_bodacc.drop('registre', axis=1)

In [60]:
# format dates
df_bodacc['dateparution'] = pd.to_datetime(df_bodacc['dateparution'])
df_bodacc['jugement_date'] = pd.to_datetime(df_bodacc['jugement_date'], format='mixed', dayfirst=True, errors='ignore')

In [61]:
df_bodacc = df_bodacc.sort_values(by=['dateparution', 'SIREN'])

In [62]:
len(df_bodacc)

2451404

In [63]:
len(df_bodacc.SIREN.unique())

900242

In [64]:
# keep only relevant categories
df_bodacc = df_bodacc.loc[(df_bodacc['jugement_famille'] != 'Rétractation sur tierce opposition') & 
                          (df_bodacc['jugement_famille'] != 'Loi de 1967') &
                          (df_bodacc['jugement_famille'] != "Arrêt de la Cour d'Appel")]

In [65]:
# convert to simple sub categories of judgements
df_bodacc.loc[(df_bodacc['jugement_famille'] == 'Jugement prononçant') & (df_bodacc['jugement_nature'] != 'Jugement de conversion en liquidation judiciaire'), 'jugement_nature'] = 'Autre jugement'
df_bodacc.loc[(df_bodacc['jugement_famille'] == 'Jugement de clôture') & (df_bodacc['jugement_nature'] != "Jugement de clôture pour insuffisance d'actif"), 'jugement_nature'] = 'Autre jugement'
df_bodacc.loc[(df_bodacc['jugement_famille'] == "Jugement d'ouverture") & ((df_bodacc['jugement_nature'] != "Jugement d'ouverture de liquidation judiciaire") & (df_bodacc['jugement_nature'] != "Jugement d'ouverture d'une procédure de redressement judiciaire")), 'jugement_nature'] = 'Autre jugement'
df_bodacc.loc[(df_bodacc['jugement_famille'] == "Avis de dépôt") & (df_bodacc['jugement_nature'] != "Dépôt de l'état des créances"), 'jugement_nature'] = 'Autre jugement'

In [66]:
# simplify jugemetn nature for ouverture
df_bodacc.loc[df_bodacc['jugement_nature'] == "Jugement d'ouverture de liquidation judiciaire", "jugement_famille"] = "Jugement d'ouverture de liquidation"
df_bodacc.loc[df_bodacc['jugement_nature'] == "Jugement d'ouverture d'une procédure de redressement judiciaire", "jugement_famille"] = "Jugement d'ouverture de redressement"
df_bodacc.loc[df_bodacc['jugement_nature'] == "Autre jugement", "jugement_famille"] = "Jugement d'ouverture autre"

In [67]:
len(df_bodacc)

2443796

In [68]:
df_bodacc = df_bodacc[df_bodacc.numerodepartement.isin(['75', '77', '78', '91', '92', '93', '94', '95'])]

In [69]:
len(df_bodacc)

481844

### Format BODACC liquidation

In [77]:
df_bodacc_liq = df_bodacc[df_bodacc.jugement_famille == "Jugement d'ouverture de liquidation"].drop_duplicates('SIREN',
                                                                                                               keep='last')

In [78]:
siren_date = df_bodacc[(df_bodacc.dateparution > '2020-01-01')]
# format other type of date TO FIX
siren_date['year'] = siren_date['dateparution'].apply(lambda x: str(x)[:4])
siren_date = siren_date[siren_date['year'].isin(['2020', '2021', '2022', '2023'])]

C:\Users\RémiJAYLET\AppData\Local\Temp\ipykernel_15672\1735739652.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  siren_date['year'] = siren_date['dateparution'].apply(lambda x: str(x)[:4])


In [79]:
df_bodacc_liq = df_bodacc_liq[df_bodacc_liq.SIREN.isin(list(siren_date.SIREN))]

In [80]:
df_bodacc_liq

,id,dateparution,numeroannonce,numerodepartement,region_code,tribunal,commercant,ville,jugement_nature,jugement_famille,jugement_date,jugement_complement,SIREN
2560262,A200800932929,2008-01-06,2929,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,ETS JEAN GUERIN,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,5 mai 2008,"Jugement prononçant la liquidation judiciaire,...",592042519
2448545,A200802001919,2008-02-11,1919,92,11.0,TRIBUNAL DE COMMERCE DE NANTERRE,ARTS-IMMO,Rueil-Malmaison,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,16 octobre 2008,"Jugement prononçant la liquidation judiciaire,...",397994914
2167934,A200800373636,2008-02-29,3636,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,SYLANDCO,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,10 janvier 2008,"Jugement prononçant la liquidation judiciaire,...",481753861
2562729,A200801361683,2008-03-08,1683,93,11.0,TRIBUNAL DE GRANDE INSTANCE DE BOBIGNY,"PROSZENUCK, Sylvie",Saint-Denis,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,3 juillet 2008,Jugement d'ouverture de la liquidation judicia...,384262960
2158501,A200800466080,2008-03-18,6080,93,11.0,TRIBUNAL DE COMMERCE DE BOBIGNY,"BARBAR, Ibrahim",Pantin,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 février 2008,"Jugement prononçant la liquidation judiciaire,...",450121975
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2730633,A202301002293,2023-05-25,2293,92,11.0,Greffe du Tribunal de Commerce de Nanterre,NAS 92,Asnières-sur-Seine,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-16,"Jugement prononçant la liquidation judiciaire,...",893263277
2510034,A202301002292,2023-05-25,2292,92,11.0,Greffe du Tribunal de Commerce de Nanterre,MURS EN COULEUR,Nanterre,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-16,"Jugement prononçant la liquidation judiciaire,...",893486308
2669859,A202301002213,2023-05-25,2213,77,11.0,Greffe du Tribunal de Commerce de Melun,ISOBAT,Vaux-le-Pénil,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-15,"Jugement prononçant la liquidation judiciaire,...",899286405
2509860,A202301002158,2023-05-25,2158,77,11.0,Greffe du Tribunal de Commerce de Meaux,BFCG Immo,Serris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-09,Jugement prononçant la liquidation judiciaire ...,899693543


In [81]:
cloture_list = list(df_bodacc[df_bodacc.jugement_famille == "Jugement de clôture"].drop_duplicates('SIREN', keep='last').SIREN)

In [82]:
df_bodacc_liq = df_bodacc_liq[~df_bodacc_liq['SIREN'].isin(cloture_list)]

In [83]:
len(df_bodacc_liq)

13089

In [84]:
df_bodacc_liq.head(3)

,id,dateparution,numeroannonce,numerodepartement,region_code,tribunal,commercant,ville,jugement_nature,jugement_famille,jugement_date,jugement_complement,SIREN
2448545,A200802001919,2008-02-11,1919,92,11.0,TRIBUNAL DE COMMERCE DE NANTERRE,ARTS-IMMO,Rueil-Malmaison,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,16 octobre 2008,"Jugement prononçant la liquidation judiciaire,...",397994914
2553698,A200800923221,2008-05-31,3221,92,11.0,TRIBUNAL DE COMMERCE DE NANTERRE,SHS FRANCE,Boulogne-Billancourt,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,13 mai 2008,"Jugement prononçant la liquidation judiciaire,...",493286553
2415614,A200801921873,2008-10-22,1873,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,DATASCOR,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,29 septembre 2008,"Jugement prononçant la liquidation judiciaire,...",323505743


### Import other bases

In [24]:
# get SIREN of companies that are PM
SIREN = pd.read_csv('../0_data/base_sirene/StockUniteLegale_utf8.csv', 
                    usecols=['siren', 'denominationUniteLegale', 'prenom1UniteLegale'],
                    dtype = {'siren': str})
SIREN = SIREN[SIREN.prenom1UniteLegale.isna()]

C:\Users\RémiJAYLET\AppData\Local\Temp\ipykernel_15672\706957035.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  SIREN = pd.read_csv('../0_data/base_sirene/StockUniteLegale_utf8.csv',


In [85]:
df_bodacc_liq = df_bodacc_liq[df_bodacc_liq.SIREN.isin(list(SIREN.siren))]

In [86]:
len(df_bodacc_liq)

12670

In [27]:
df_parc1 = pd.concat([pd.read_csv(i,
                                  sep=';',
                                  encoding='latin',
                                  low_memory=False,
                                  dtype={'Département (Champ géographique)' : str,
                                         'Code Direction (Champ géographique)' : str,
                                         'Code Commune (Champ géographique)' : str,
                                         'Préfixe (Références cadastrales)' : str,
                                         'Section (Références cadastrales)' : str,
                                         'N° plan (Références cadastrales)' : str}) for i in glob.glob(
    "../0_data/parcelles/Fichier des parcelles (situation 2022)-dept 01 à 61/Fichier des parcelles (situation 2022)-dpts 01 à 61/*.txt", recursive=True)])

df_parc2 = pd.concat([pd.read_csv(i, sep=';',
                                  encoding='latin',
                                  low_memory=False,
                                  dtype={'Département (Champ géographique)' : str,
                                         'Code Direction (Champ géographique)' : str,
                                         'Code Commune (Champ géographique)' : str,
                                         'Préfixe (Références cadastrales)' : str,
                                         'Section (Références cadastrales)' : str,
                                         'N° plan (Références cadastrales)' : str}) for i in glob.glob(
    "../0_data/parcelles/Fichier des parcelles (situation 2022)-dept 62 à 976/Fichier des parcelles (situation 2022)-dpts 62 à 976/*.txt", recursive=True)])

df_parc = pd.concat([df_parc1, df_parc2], axis=0)

In [28]:
df_loc = pd.concat([pd.read_csv(i,
                                sep=';',
                                encoding='latin',
                                low_memory=False,
                                dtype={'Département (Champ géographique)' : str,
                                         'Code Direction (Champ géographique)' : str,
                                         'Code Commune (Champ géographique)' : str,
                                         'Préfixe (Références cadastrales)' : str,
                                         'Section (Références cadastrales)' : str,
                                         'N° plan (Références cadastrales)' : str}) for i in glob.glob(
    "../0_data/locaux/Fichier des locaux/*.txt",
    recursive=True)])

In [29]:
SIRET = pd.read_csv('../0_data/base_sirene/StockEtablissement_utf8.csv',
                    usecols=['siren', 'siret', 'libelleVoieEtablissement', 'libelleCommuneEtablissement'],
                    dtype = {'siren': str, 'siret': str, 'libelleVoieEtablissement': str, 'libelleCommuneEtablissement': str})

### Analysis

In [87]:
df = df_bodacc_liq.copy()

In [88]:
df

,id,dateparution,numeroannonce,numerodepartement,region_code,tribunal,commercant,ville,jugement_nature,jugement_famille,jugement_date,jugement_complement,SIREN
2448545,A200802001919,2008-02-11,1919,92,11.0,TRIBUNAL DE COMMERCE DE NANTERRE,ARTS-IMMO,Rueil-Malmaison,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,16 octobre 2008,"Jugement prononçant la liquidation judiciaire,...",397994914
2553698,A200800923221,2008-05-31,3221,92,11.0,TRIBUNAL DE COMMERCE DE NANTERRE,SHS FRANCE,Boulogne-Billancourt,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,13 mai 2008,"Jugement prononçant la liquidation judiciaire,...",493286553
2415614,A200801921873,2008-10-22,1873,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,DATASCOR,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,29 septembre 2008,"Jugement prononçant la liquidation judiciaire,...",323505743
2532687,A200900191945,2009-01-28,1945,94,11.0,TRIBUNAL DE GRANDE INSTANCE DE CRETEIL,S.C.I. DE LA RENAISSANCE,Orly,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,13 janvier 2009,Jugement d'ouverture de liquidation judiciaire...,429620743
2459631,A200901391945,2009-07-23,1945,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,ACANTHE COMMUNICATION,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,25 juin 2009,"Jugement prononçant la liquidation judiciaire,...",487806507
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2730633,A202301002293,2023-05-25,2293,92,11.0,Greffe du Tribunal de Commerce de Nanterre,NAS 92,Asnières-sur-Seine,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-16,"Jugement prononçant la liquidation judiciaire,...",893263277
2510034,A202301002292,2023-05-25,2292,92,11.0,Greffe du Tribunal de Commerce de Nanterre,MURS EN COULEUR,Nanterre,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-16,"Jugement prononçant la liquidation judiciaire,...",893486308
2669859,A202301002213,2023-05-25,2213,77,11.0,Greffe du Tribunal de Commerce de Melun,ISOBAT,Vaux-le-Pénil,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-15,"Jugement prononçant la liquidation judiciaire,...",899286405
2509860,A202301002158,2023-05-25,2158,77,11.0,Greffe du Tribunal de Commerce de Meaux,BFCG Immo,Serris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-09,Jugement prononçant la liquidation judiciaire ...,899693543


#### With parcelles

In [89]:
parc_bodacc = df.merge(df_parc, left_on='SIREN',
                       right_on = 'N° SIREN (Propriétaire(s) parcelle)',
                       how='left')

In [90]:
parc_bodacc = parc_bodacc[~parc_bodacc['N° SIREN (Propriétaire(s) parcelle)'].isna()]
parc_bodacc = parc_bodacc.drop_duplicates(['SIREN', 'jugement_date', 'Département (Champ géographique)', 
                                           'Code Commune (Champ géographique)', 'Section (Références cadastrales)',
                                           'Préfixe (Références cadastrales)', 'N° plan (Références cadastrales)'])

In [91]:
parc_bodacc

,id,dateparution,numeroannonce,numerodepartement,region_code,tribunal,commercant,ville,jugement_nature,jugement_famille,jugement_date,jugement_complement,SIREN,Département (Champ géographique),Code Direction (Champ géographique),Code Commune (Champ géographique),Nom Commune (Champ géographique),Préfixe (Références cadastrales),Section (Références cadastrales),N° plan (Références cadastrales),N° de voirie (Adresse parcelle),Indice de répétition (Adresse parcelle),Code voie MAJIC (Adresse parcelle),Code voie rivoli (Adresse parcelle),Nature voie (Adresse parcelle),Nom voie (Adresse parcelle),Contenance (Caractéristiques parcelle),SUF (Evaluation SUF),Nature culture (Evaluation SUF),Contenance (Evaluation SUF),Code droit (Propriétaire(s) parcelle),N° MAJIC (Propriétaire(s) parcelle),N° SIREN (Propriétaire(s) parcelle),Groupe personne (Propriétaire(s) parcelle),Forme juridique (Propriétaire(s) parcelle),Forme juridique abrégée (Propriétaire(s) parcelle),Dénomination (Propriétaire(s) parcelle)
19,A201200192285,2012-01-27,2285,93,11.0,TRIBUNAL DE COMMERCE DE BOBIGNY,FONCIERE RICHELIEU,Montreuil,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,12 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",333697795,93,0,027,LA COURNEUVE,,O,0343,NaN,NaN,180.0,7125,RUE,PASCAL,690.0,NaN,S,690.0,P,PBBLQ7,333697795,0,5710,SAS,FONCIERE RICHELIEU
20,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,1,449,ORCHIES,,AA,0253,NaN,NaN,134.0,0031,RUE,ATTILIO BERNASCONI,8639.0,NaN,AG,8639.0,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE
21,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,1,449,ORCHIES,,AA,0254,NaN,NaN,134.0,0031,RUE,ATTILIO BERNASCONI,812.0,NaN,AB,812.0,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE
22,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,1,449,ORCHIES,,AA,0255,NaN,NaN,134.0,0031,RUE,ATTILIO BERNASCONI,1042.0,NaN,AG,1042.0,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE
23,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,1,449,ORCHIES,,AA,0256,NaN,NaN,134.0,0031,RUE,ATTILIO BERNASCONI,6467.0,NaN,AB,6467.0,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13424,A202301002246,2023-05-25,2246,78,11.0,Greffe du Tribunal de Commerce de Versailles,SH L'ESPERANCE,Saint-Père,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-16,"Jugement prononçant la liquidation judiciaire,...",820222214,89,0,364,SAINT PERE,,AB,0721,NaN,NaN,183.0,B185,NaN,LE VILLAGE,1812.0,NaN,AG,1812.0,P,PBCWT4,820222214,0,5710,SAS,SH LESPERANCE
13425,A202301002246,2023-05-25,2246,78,11.0,Greffe du Tribunal de Commerce de Versailles,SH L'ESPERANCE,Saint-Père,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-16,"Jugement prononçant la liquidation judiciaire,...",820222214,89,0,364,SAINT PERE,,AB,0794,NaN,NaN,69.0,B070,NaN,L ENFERMIE,35.0,NaN,S,35.0,P,PBCWT4,820222214,0,5710,SAS,SH LESPERANCE
13426,A202301002246,2023-05-25,2246,78,11.0,Greffe du Tribunal de Commerce de Versailles,SH L'ESPERANCE,Saint-Père,Jugement d'ou

In [92]:
print('Number of parcelles concerned :', len(parc_bodacc))

Number of parcelles concerned : 905


In [109]:
parc_bodacc

,id,dateparution,numeroannonce,numerodepartement,region_code,tribunal,commercant,ville,jugement_nature,jugement_famille,jugement_date,jugement_complement,SIREN,Département (Champ géographique),Code Direction (Champ géographique),Code Commune (Champ géographique),Nom Commune (Champ géographique),Préfixe (Références cadastrales),Section (Références cadastrales),N° plan (Références cadastrales),N° de voirie (Adresse parcelle),Indice de répétition (Adresse parcelle),Code voie MAJIC (Adresse parcelle),Code voie rivoli (Adresse parcelle),Nature voie (Adresse parcelle),Nom voie (Adresse parcelle),Contenance (Caractéristiques parcelle),SUF (Evaluation SUF),Nature culture (Evaluation SUF),Contenance (Evaluation SUF),Code droit (Propriétaire(s) parcelle),N° MAJIC (Propriétaire(s) parcelle),N° SIREN (Propriétaire(s) parcelle),Groupe personne (Propriétaire(s) parcelle),Forme juridique (Propriétaire(s) parcelle),Forme juridique abrégée (Propriétaire(s) parcelle),Dénomination (Propriétaire(s) parcelle),parcelle_id
19,A201200192285,2012-01-27,2285,93,11.0,TRIBUNAL DE COMMERCE DE BOBIGNY,FONCIERE RICHELIEU,Montreuil,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,12 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",333697795,93,0,027,LA COURNEUVE,0000,O,0343,NaN,NaN,180.0,7125,RUE,PASCAL,690.0,NaN,S,690.0,P,PBBLQ7,333697795,0,5710,SAS,FONCIERE RICHELIEU,930270000O0343
20,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,1,449,ORCHIES,000,AA,0253,NaN,NaN,134.0,0031,RUE,ATTILIO BERNASCONI,8639.0,NaN,AG,8639.0,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE,59449000AA0253
21,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,1,449,ORCHIES,000,AA,0254,NaN,NaN,134.0,0031,RUE,ATTILIO BERNASCONI,812.0,NaN,AB,812.0,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE,59449000AA0254
22,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,1,449,ORCHIES,000,AA,0255,NaN,NaN,134.0,0031,RUE,ATTILIO BERNASCONI,1042.0,NaN,AG,1042.0,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE,59449000AA0255
23,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,1,449,ORCHIES,000,AA,0256,NaN,NaN,134.0,0031,RUE,ATTILIO BERNASCONI,6467.0,NaN,AB,6467.0,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE,59449000AA0256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13424,A202301002246,2023-05-25,2246,78,11.0,Greffe du Tribunal de Commerce de Versailles,SH L'ESPERANCE,Saint-Père,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-16,"Jugement prononçant la liquidation judiciaire,...",820222214,89,0,364,SAINT PERE,000,AB,0721,NaN,NaN,183.0,B185,NaN,LE VILLAGE,1812.0,NaN,AG,1812.0,P,PBCWT4,820222214,0,5710,SAS,SH LESPERANCE,89364000AB0721
13425,A202301002246,2023-05-25,2246,78,11.0,Greffe du Tribunal de Commerce de Versailles,SH L'ESPERANCE,Saint-Père,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-16,"Jugement prononçant la liquidation judiciaire,...",820222214,89,0,364,SAINT PERE,000,AB,0794,NaN,NaN,69.0,B070,NaN,L ENFERMIE,35.0,NaN,S,35.0,P,PBCWT4,820222214,0,5710,SAS,SH LESPERANCE,

In [111]:
parc_bodacc.sort_values(.to_csv('20230525_propriete_parcelles_bodacc_idf.csv', sep=';', encoding='utf-8-sig')

In [93]:
print('Number of companies concerned :', len(parc_bodacc.SIREN.unique()))

Number of companies concerned : 139


In [94]:
parc_bodacc['Préfixe (Références cadastrales)'] = parc_bodacc['Section (Références cadastrales)'].apply(lambda x: '000' if len(x) == 2 else '0000')
parc_bodacc['parcelle_id'] = parc_bodacc['Département (Champ géographique)'] + parc_bodacc['Code Commune (Champ géographique)'] + parc_bodacc['Préfixe (Références cadastrales)'] + parc_bodacc['Section (Références cadastrales)']  + parc_bodacc['N° plan (Références cadastrales)']

In [95]:
list_cities = ['ALFORTVILLE', 'CHAMPIGNY SUR MARNE', 'FONTENAY SOUS BOIS',
               'ST MAUR DES FOSSES', 'ASNIERES-SUR-SEINE', 'CLICHY', 'LIVRY GARGAN']

parc_bodacc[parc_bodacc['Nom Commune (Champ géographique)'].isin(list_cities)]

,id,dateparution,numeroannonce,numerodepartement,region_code,tribunal,commercant,ville,jugement_nature,jugement_famille,jugement_date,jugement_complement,SIREN,Département (Champ géographique),Code Direction (Champ géographique),Code Commune (Champ géographique),Nom Commune (Champ géographique),Préfixe (Références cadastrales),Section (Références cadastrales),N° plan (Références cadastrales),N° de voirie (Adresse parcelle),Indice de répétition (Adresse parcelle),Code voie MAJIC (Adresse parcelle),Code voie rivoli (Adresse parcelle),Nature voie (Adresse parcelle),Nom voie (Adresse parcelle),Contenance (Caractéristiques parcelle),SUF (Evaluation SUF),Nature culture (Evaluation SUF),Contenance (Evaluation SUF),Code droit (Propriétaire(s) parcelle),N° MAJIC (Propriétaire(s) parcelle),N° SIREN (Propriétaire(s) parcelle),Groupe personne (Propriétaire(s) parcelle),Forme juridique (Propriétaire(s) parcelle),Forme juridique abrégée (Propriétaire(s) parcelle),Dénomination (Propriétaire(s) parcelle),parcelle_id
273,A201602462390,2016-12-18,2390,94,11.0,TRIBUNAL DE COMMERCE DE CRÉTEIL,CABINET LAMBERT,Fontenay-sous-Bois,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2016-12-07,"Jugement prononçant la liquidation judiciaire,...",342376928,94,0,033,FONTENAY SOUS BOIS,000,BQ,0002,77.0,NaN,342.0,7140,RUE,PASTEUR,836.0,NaN,S,836.0,G,PBBBF9,342376928,0,5710,SAS,CABINET LAMBERT,94033000BQ0002
274,A201602462390,2016-12-18,2390,94,11.0,TRIBUNAL DE COMMERCE DE CRÉTEIL,CABINET LAMBERT,Fontenay-sous-Bois,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2016-12-07,"Jugement prononçant la liquidation judiciaire,...",342376928,94,0,033,FONTENAY SOUS BOIS,0000,R,0015,20.0,NaN,185.0,3160,AV,ERNEST RENAN,211.0,NaN,S,211.0,S,PBBBF9,342376928,0,5710,SAS,CABINET LAMBERT,940330000R0015
2769,A202100024350,2021-01-05,4350,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,CABINET ROLET-BONTEMPS,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2020-12-17,"Jugement prononçant la liquidation judiciaire,...",331153676,92,1,024,CLICHY,0000,Q,0068,56.0,NaN,110.0,7130,RUE,DE PARIS,288.0,NaN,S,288.0,G,PBFS74,331153676,0,5710,SAS,CABINET ROLET BONTEMPS,920240000Q0068
2770,A202100024350,2021-01-05,4350,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,CABINET ROLET-BONTEMPS,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2020-12-17,"Jugement prononçant la liquidation judiciaire,...",331153676,92,1,024,CLICHY,0000,Q,0069,71.0,NaN,70.0,4420,RUE,HENRI BARBUSSE,354.0,NaN,S,354.0,G,PBFS74,331153676,0,5710,SAS,CABINET ROLET BONTEMPS,920240000Q0069
2771,A202100024350,2021-01-05,4350,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,CABINET ROLET-BONTEMPS,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2020-12-17,"Jugement prononçant la liquidation judiciaire,...",331153676,92,1,024,CLICHY,0000,Q,0148,75.0,NaN,70.0,4420,RUE,HENRI BARBUSSE,280.0,NaN,S,280.0,G,PBFS74,331153676,0,5710,SAS,CABINET ROLET BONTEMPS,920240000Q0148
2772,A202100024350,2021-01-05,4350,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,CABINET ROLET-BONTEMPS,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2020-12-17,"Jugement prononçant la liquidation judiciaire,...",331153676,92,1,024,CLICHY,0000,Q,0179,75.0,NaN,70.0,4420,RUE,HENRI BARBUSSE,290.0,NaN,S,290.0,G,PBFS74,331153676,0,5710,SAS,CABINET ROLET BONTEMPS,920240000Q0179
3415,A202101032527,2021-05-28,2527,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,PROGESCO,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2021-05-12,Jugement prononçant la liquidation judiciaire ...,682024542,92,1,004,ASNIERES-SUR-SEINE,000,AQ,0163,17.0,NaN,304.0,6660,RUE,DE NORMANDIE,210.0,NaN,S,210.0,S,PBBPVS,682024542,0,5710,SAS,PROGESCO SA,92004000AQ0163
3638,A202101232874,2021-06-25,2874,94,11.0,TRIBUNAL DE COMMERCE DE CRÉTEIL,ADB PATRIMOINE,Saint-Maur-des-Fossés,Jugemen

#### With locaux

In [96]:
loc_bodacc = df.merge(df_loc,
                      left_on='SIREN',
                      right_on = 'N° SIREN (Propriétaire(s) du local)',
                      how='left')

In [97]:
loc_bodacc = loc_bodacc[~loc_bodacc['N° SIREN (Propriétaire(s) du local)'].isna()]
loc_bodacc = loc_bodacc.drop_duplicates(['SIREN', 'jugement_date', 'Département (Champ géographique)', 
                                         'Code Commune (Champ géographique)', 'Section (Références cadastrales)',
                                         'Préfixe (Références cadastrales)', 'N° plan (Références cadastrales)'])

In [98]:
loc_bodacc

,id,dateparution,numeroannonce,numerodepartement,region_code,tribunal,commercant,ville,jugement_nature,jugement_famille,jugement_date,jugement_complement,SIREN,Département (Champ géographique),Code Direction (Champ géographique),Code Commune (Champ géographique),Nom Commune (Champ géographique),Préfixe (Références cadastrales),Section (Références cadastrales),N° plan (Références cadastrales),Bâtiment (Identification du local),Entrée (Identification du local),Niveau (Identification du local),Porte (Identification du local),N° voirie (Adresse du local),Indice de répétition (Adresse du local),Code voie MAJIC (Adresse du local),Code voie rivoli (Adresse du local),Nature voie (Adresse du local),Nom voie (Adresse du local),Code droit (Propriétaire(s) du local),N° MAJIC (Propriétaire(s) du local),N° SIREN (Propriétaire(s) du local),Groupe personne (Propriétaire(s) du local),Forme juridique (Propriétaire(s) du local),Forme juridique abrégée (Propriétaire(s) du local),Dénomination (Propriétaire(s) du local),parcelle_id
2,A200801921873,2008-10-22,1873,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,DATASCOR,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,29 septembre 2008,"Jugement prononçant la liquidation judiciaire,...",323505743,75,5,119,PARIS 19,000,AE,0001,A,03,0.0,1001.0,122.0,NaN,120.0,0517,RUE,D AUBERVILLIERS,P,PBDDGQ,323505743,0,5499,SARL,DATASCOR,75119000AE0001
9,A201002152048,2010-05-11,2048,93,11.0,TRIBUNAL DE COMMERCE DE BOBIGNY,DOCKS DE LA MALTOURNEE ETS F GUEGUEN ET CIE,Neuilly-sur-Marne,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,19 octobre 2010,"Jugement prononçant la liquidation judiciaire,...",558201760,93,0,066,SAINT DENIS,000,BY,0151,A,01,0.0,1001.0,29.0,B,81.0,0023,RUE,AMBROISE CROIZAT,P,PBBKFM,558201760,0,5599,SA,DOCKS LA MALTOURNEE ETS F GUEGUEN ET CIE,93066000BY0151
20,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,2,459,PETITE FORET,000,AB,0248,A,01,0.0,1001.0,20.0,P,135.0,0886,VC,RES SIMONE DE BEAUVOIR,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE,59459000AB0248
22,A201201262179,2012-03-07,2179,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,BAO,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,6 juin 2012,"Jugement prononçant la liquidation judiciaire,...",508608593,26,0,057,BOURG-DE-PEAGE,000,AH,0821,01,01,0.0,3.0,6.0,NaN,192.0,0745,ALL,DE PROVENCE,P,PBCXH3,508608593,0,5499,SARL,BAO,26057000AH0821
65,A201401342719,2014-07-16,2719,95,11.0,TRIBUNAL DE COMMERCE DE PONTOISE,SMS ENTREPOT,Persan,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,30 juin 2014,Jugement prononçant la liquidation judiciaire ...,532252228,95,0,487,PERSAN,000,AH,0017,I,01,0.0,1001.0,39.0,NaN,82.0,0170,RUE,DU DOCTEUR TOUATI,P,PBD6JQ,532252228,0,5499,SARL,SMS ENTREPOT,95487000AH0017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13672,A202301002246,2023-05-25,2246,78,11.0,Greffe du Tribunal de Commerce de Versailles,SH L'ESPERANCE,Saint-Père,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-16,"Jugement prononçant la liquidation judiciaire,...",820222214,89,0,364,SAINT PERE,000,AB,0010,A,01,0.0,1001.0,25.0,NaN,200.0,0160,GR,GRANDE RUE,P,PBCWT4,820222214,0,5710,SAS,SH LESPERANCE,89364000AB0010
13673,A202301002246,2023-05-25,2246,78,11.0,Greffe du Tribunal de Commerce de Versailles,SH L'ESPERANCE,Saint-Père,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-16,"Jugement prononçant la liquidation judiciaire,...",820222214,89,0,364,SAINT PERE,000,AB,0502,A,01,0.0,1001.0,22.0,NaN,200.0,0160,GR,GRANDE RUE,P,PBCWT4,820222214,0,5710

In [99]:
print('Number of locaux concerned :', len(loc_bodacc))

Number of locaux concerned : 404


In [100]:
loc_bodacc['Préfixe (Références cadastrales)'] = loc_bodacc['Section (Références cadastrales)'].apply(lambda x: '000' if len(x) == 2 else '0000')
df_loc['Préfixe (Références cadastrales)'] = df_loc['Section (Références cadastrales)'].apply(lambda x: '000' if len(str(x)) == 2 else '0000')

loc_bodacc['parcelle_id'] = loc_bodacc['Département (Champ géographique)'] + loc_bodacc['Code Commune (Champ géographique)']+ loc_bodacc['Préfixe (Références cadastrales)'] + loc_bodacc['Section (Références cadastrales)']  + loc_bodacc['N° plan (Références cadastrales)']
df_loc['parcelle_id'] = df_loc['Département (Champ géographique)'] + df_loc['Code Commune (Champ géographique)'] + df_loc['Préfixe (Références cadastrales)']  + df_loc['Section (Références cadastrales)'] + df_loc['N° plan (Références cadastrales)']

In [101]:
df_loc_perimetre = df_loc[df_loc['parcelle_id'].isin(loc_bodacc['parcelle_id'].unique())]
count_per_loc = df_loc_perimetre[['parcelle_id','N° SIREN (Propriétaire(s) du local)']].groupby('parcelle_id').count()
print('Proportion of locaux  where there is unique SIREN in it: ',
      round(100*len(count_per_loc[count_per_loc['N° SIREN (Propriétaire(s) du local)']!=1])/len(count_per_loc),2), '%' )

Proportion of locaux  where there is unique SIREN in it:  81.45 %


#### With SIRET

In [102]:
siret_bodacc = df.merge(SIRET, left_on='SIREN',
                               right_on = 'siren',
                               how='left')

In [103]:
len(siret_bodacc)

25969

In [104]:
siret_bodacc_loc = siret_bodacc.merge(loc_bodacc, how='left',
                                      left_on=['siren', 'libelleVoieEtablissement', 'libelleCommuneEtablissement'],
                                      right_on=['N° SIREN (Propriétaire(s) du local)',
                                                'Nom voie (Adresse du local)', 'Nom Commune (Champ géographique)'])

In [105]:
siret_bodacc_loc = siret_bodacc_loc[['siren', 'siret', 'libelleVoieEtablissement', 'libelleCommuneEtablissement',
                  'Code Commune (Champ géographique)', 'Nom Commune (Champ géographique)',
                  'Bâtiment (Identification du local)',
                  'Entrée (Identification du local)', 'Niveau (Identification du local)',
                  'Porte (Identification du local)', 'N° voirie (Adresse du local)',
                  'Indice de répétition (Adresse du local)',
                  'Code voie MAJIC (Adresse du local)', 'Code voie rivoli (Adresse du local)', 'Nature voie (Adresse du local)',
                  'Nom voie (Adresse du local)', 'Code droit (Propriétaire(s) du local)',
                  'N° MAJIC (Propriétaire(s) du local)',
                  'N° SIREN (Propriétaire(s) du local)',
                  'Groupe personne (Propriétaire(s) du local)',
                  'Forme juridique (Propriétaire(s) du local)',
                  'Forme juridique abrégée (Propriétaire(s) du local)',
                  'Dénomination (Propriétaire(s) du local)', 'parcelle_id']]
siret_bodacc_loc = siret_bodacc_loc[~siret_bodacc_loc['parcelle_id'].isna()]

In [106]:
len(siret_bodacc_loc)

30

In [107]:
siret_bodacc_loc

,siren,siret,libelleVoieEtablissement,libelleCommuneEtablissement,Code Commune (Champ géographique),Nom Commune (Champ géographique),Bâtiment (Identification du local),Entrée (Identification du local),Niveau (Identification du local),Porte (Identification du local),N° voirie (Adresse du local),Indice de répétition (Adresse du local),Code voie MAJIC (Adresse du local),Code voie rivoli (Adresse du local),Nature voie (Adresse du local),Nom voie (Adresse du local),Code droit (Propriétaire(s) du local),N° MAJIC (Propriétaire(s) du local),N° SIREN (Propriétaire(s) du local),Groupe personne (Propriétaire(s) du local),Forme juridique (Propriétaire(s) du local),Forme juridique abrégée (Propriétaire(s) du local),Dénomination (Propriétaire(s) du local),parcelle_id
6,323505743,32350574300019,D AUBERVILLIERS,PARIS 19,119,PARIS 19,A,03,0.0,1001.0,122.0,NaN,120.0,0517,RUE,D AUBERVILLIERS,P,PBDDGQ,323505743,0,5499,SARL,DATASCOR,75119000AE0001
139,532252228,53225222800012,DU DOCTEUR TOUATI,PERSAN,487,PERSAN,I,01,0.0,1001.0,39.0,NaN,82.0,0170,RUE,DU DOCTEUR TOUATI,P,PBD6JQ,532252228,0,5499,SARL,SMS ENTREPOT,95487000AH0017
775,415107739,41510773900021,DE LAGNY,MONTREUIL,048,MONTREUIL,A,01,0.0,1002.0,28.0,NaN,242.0,5110,RUE,DE LAGNY,P,PBC883,415107739,0,5499,SARL,SANITAIRE PLOMBERIE CHAUFFAGE CLIMATISAT,93048000BF0123
2433,622038909,62203890900017,KLEBER,PARIS 16,116,PARIS 16,A,01,0.0,7001.0,59.0,NaN,496.0,5159,AV,KLEBER,P,PBPND7,622038909,0,5499,SARL,ROAL CHAUSSURES REPARATIONS,75116000FL0058
3485,522270503,52227050300017,RASPAIL,ALFORTVILLE,002,ALFORTVILLE,A,01,0.0,7.0,32.0,NaN,133.0,8120,RUE,RASPAIL,P,PBFTCC,522270503,0,6540,SCI,SCI MJEN,940020000S0292
5513,324895788,32489578800010,BARBES,PARIS 18,118,PARIS 18,A,01,81.0,2001.0,42.0,NaN,104.0,0662,BD,BARBES,P,PBG66L,324895788,0,5499,SARL,L ETOILE DE TUNIS,75118000BT0106
5558,799439377,79943937700018,PIERRE BROSSOLETTE,GARGENVILLE,267,GARGENVILLE,A,01,0.0,1002.0,27.0,NaN,184.0,1300,RUE,PIERRE BROSSOLETTE,P,PBGG4H,799439377,0,6540,SCI,SOCIETE CIVILE IMMOBILIERE CMM ET PAR ABREVIAT...,78267000AC0002
5585,488601113,48860111300019,JEAN DE LA FONTAINE,PARIS 16,116,PARIS 16,B,02,5.0,1001.0,85.0,NaN,498.0,5249,RUE,JEAN DE LA FONTAINE,P,PBMPS6,488601113,0,5710,SAS,BALSA-COMPOSITES,75116000BQ0100
7351,451217681,45121768100019,DES CARRIERS ITALIENS,GRIGNY,286,GRIGNY,A,01,0.0,1001.0,16.0,NaN,242.0,0147,RUE,DES CARRIERS ITALIENS,P,PBCJJB,451217681,0,6540,SCI,MOULIN STAR,91286000AP0137
7360,622820447,62282044700036,FONTAINE L EPINE,MORTEAU,411,MORTEAU,A,01,0.0,1001.0,4.0,NaN,141.0,0183,RUE,FONTAINE L EPINE,P,PBCH34,622820447,0,5710,SAS,SOCIETE PAGNOT ET COMPAGNIE,25411000AE0304
